In [1]:
import pandas as pd

In [2]:
# read db export
df = pd.read_csv("exp_db.csv").dropna(how="all")
# select where to start and stop generating
df = df.loc[df["Short name"].apply(lambda x: 1424 < int(x[2:]) < 1525)]
df

,Short name,Date,Data set ID,Split ID,Estimator,Features,Tags
326,JG1425,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_80,FFN,OHE,NaN
327,JG1426,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_80,XGB,FP,NaN
328,JG1427,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_80,XGB,"FP, RDKit",NaN
329,JG1428,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_80,D-MPNN,CGR,NaN
330,JG1429,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_80,D-MPNN,"CGR, RDKit",NaN
331,JG1430,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_70,FFN,OHE,NaN
332,JG1431,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_70,XGB,FP,NaN
333,JG1432,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_70,XGB,"FP, RDKit",NaN
334,JG1433,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_70,D-MPNN,CGR,NaN
335,JG1434,NaN,synferm_dataset_2023-12-20_39486records,synferm_dataset_2023-12-20_3D_split_70,D-MPNN,"CGR, RDKit",NaN


In [3]:
def get_features(s):
    if s == "CGR":
        return "None"
    s = s.replace(",", "")
    s = s.replace("CGR", "")
    s = s.strip()
    return s
        

In [4]:
for i, row in df.iterrows():
    model = row["Estimator"].replace("-", "")
    global_features = get_features(row["Features"])
    exp_id = row["Short name"]
    split = row["Split ID"]
    tags = "_".join([row["Split ID"].split("_")[-3], row["Split ID"].split("_")[-1]])  # may sometimes fail
    print(f"nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_{model}.yaml --data-path data/synferm_dataset_2023-12-20_39486records.csv --split-indices data/splits/{split}/ --smiles-columns reaction_smiles_atom_mapped --label-columns binary_A binary_B binary_C --global-features {global_features} --global-features-file None --task multilabel --run-test --hparam-optimization --hparam-config-path config/hparam_bounds_{model}.yaml --hparam-n-iter 20 --experiment-id {exp_id} --tags {tags}")
    print()


nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_FFN.yaml --data-path data/synferm_dataset_2023-12-20_39486records.csv --split-indices data/splits/synferm_dataset_2023-12-20_3D_split_80/ --smiles-columns reaction_smiles_atom_mapped --label-columns binary_A binary_B binary_C --global-features OHE --global-features-file None --task multilabel --run-test --hparam-optimization --hparam-config-path config/hparam_bounds_FFN.yaml --hparam-n-iter 20 --experiment-id JG1425 --tags 3D_80

nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_XGB.yaml --data-path data/synferm_dataset_2023-12-20_39486records.csv --split-indices data/splits/synferm_dataset_2023-12-20_3D_split_80/ --smiles-columns reaction_smiles_atom_mapped --label-columns binary_A binary_B binary_C --global-features FP --global-features-file 